# PR0402 - Datos del clima

In [1]:
!hdfs dfs -mkdir /datos_clima

In [ ]:
!hdfs dfs -put city_temperature.csv /datos_clima

### Datos reducidos para test

He creado una versión muy reducida del csv para poder realizar pruebas de forma mucho más rápida.

In [1]:
!hdfs dfs -put test_city_temperature.csv /datos_clima

In [2]:
!hdfs dfs -ls /datos_clima

Found 2 items
-rw-r--r--   3 root supergroup  140600832 2025-11-18 09:36 /datos_clima/city_temperature.csv
-rw-r--r--   3 root supergroup       2905 2025-11-19 11:02 /datos_clima/test_city_temperature.csv


### Ejercicio 1: temperatura máxima por ciudad 

In [58]:
%%writefile mapper.py
#!/usr/bin/env python3

import sys, csv

#reader = csv.DictReader(sys.stdin)
#for line in reader:
#    print(f"{line['City']}\t{line['AvgTemperature']}")

entrada = sys.stdin
i = 0
city_index = 0
temp_index = 0

for line in entrada:
    data = line.strip().split(',')
    # En la primera línea recoge los índices que usaremos después
    if i == 0:
        city_index = data.index('City')
        temp_index = data.index('AvgTemperature')
    else:
        print(f"{data[city_index]}\t{data[temp_index]}")
        
    i += 1
    
    

Overwriting mapper.py


In [67]:
%%writefile reducer.py
#!/usr/bin/env python3

import sys, math

current_city = None
max_temp = -math.inf
#cities = set()

for line in sys.stdin:   
    city, temp = line.strip().split("\t", 1)
    temp = float(temp)

    if current_city == city:
        #cities.add(city)
        if temp > max_temp:
            max_temp = temp
    else:
        if current_city:
                print(f"{current_city}: {max_temp}")
        max_temp = -math.inf
        current_city = city
        #cities = {city}
        
if current_city:
    print(f"{current_city}: {max_temp}")

Overwriting reducer.py


In [ ]:
!hdfs dfs -rmdir /salida_temp
!hadoop jar \
/usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-3.4.0.jar \
-file mapper.py \
-file reducer.py \
-mapper mapper.py \
-reducer reducer.py \
-input /datos_clima/test_city_temperature.csv \
-output /salida_temp

rmdir: `/salida_temp': No such file or directory
2025-11-19 11:43:12,145 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [mapper.py, reducer.py, /tmp/hadoop-unjar2126178414716862642/] [] /tmp/streamjob5285207056610937163.jar tmpDir=null
2025-11-19 11:43:13,191 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.5:8032
2025-11-19 11:43:13,419 INFO client.DefaultNoHARMFailoverProxyProvider: Connecting to ResourceManager at yarnmanager/172.19.0.5:8032
2025-11-19 11:43:13,857 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/root/.staging/job_1763549823418_0005
2025-11-19 11:43:15,029 INFO mapred.FileInputFormat: Total input files to process : 1
2025-11-19 11:43:15,292 INFO mapreduce.JobSubmitter: number of splits:2
2025-11-19 11:43:16,296 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1763549823418_0005
2025-11-19 11:43

In [68]:
!cat city_temperature.csv | python3 mapper.py | sort | python3 reducer.py

Abidjan: 88.6
Abilene: 94.2
Abu Dhabi: 107.3
Addis Ababa: 77.0
Akron Canton: 86.1
Albany: 88.0
Albuquerque: 89.4
Algiers: 96.6
Allentown: 91.1
Almaty: 90.9
Amarillo: 92.5
Amman: 95.4
Amsterdam: 85.5
Anchorage: 75.3
Ankara: 87.9
Antananarivo: 78.5
Ashabad: 102.2
Asheville: 85.1
Athens: 94.3
Atlanta: 92.8
Atlantic City: 93.3
Auckland: 75.4
Austin: 94.5
Baltimore: 91.9
Bangkok: 93.0
Bangui: 93.7
Banjul: 93.6
Barcelona: 86.6
Baton Rouge: 90.2
Beijing: 92.9
Beirut: 91.1
Belfast: 77.6
Belgrade: 91.9
Belize City: 92.9
Bern: 83.7
Bilbao: 94.6
Billings: 90.6
Birmingham: 91.0
Bishkek: 91.1
Bismarck: 91.7
Bissau: 100.1
Bogota: 66.7
Boise: 94.2
Bombay (Mumbai): 92.6
Bonn: 86.9
Bordeaux: 88.8
Boston: 90.7
Brasilia: 87.7
Bratislava: 85.5
Brazzaville: 88.7
Bridgeport: 87.0
Bridgetown: 88.0
Brisbane: 87.3
Brownsville: 91.2
Brussels: 85.4
Bucharest: 91.4
Budapest: 88.2
Buenos Aires: 90.9
Buffalo: 84.4
Bujumbura: 89.1
Burlington: 88.8
Cairo: 100.2
Calcutta: 96.8
Calgary: 79.1
Canberra: 93.2
Capetown: 83